# Evolver Loop 13 Analysis

## Issue: Submission failed with 'Overlapping trees in group 004'

Our local validation passed but Kaggle rejected. Need to investigate the N=4 case specifically.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 50  # Higher precision
SCALE = 10**18

print("Setup complete")

Setup complete


In [2]:
# Load the submission
df = pd.read_csv('/home/code/experiments/014_nfp_local_search/submission.csv')
print(f"Submission shape: {df.shape}")

# Extract N=4 trees
n4_trees = df[df['id'].str.startswith('004_')].copy()
print(f"\nN=4 trees:")
print(n4_trees)

Submission shape: (20100, 4)

N=4 trees:
      id                     x                     y                  deg
6  004_0  s-0.3247477895908756   s0.1321099780911856   s156.3706221456364
7  004_1   s0.3153543462411342   s0.1321099780664757   s156.3706222692641
8  004_2   s0.3247477895908756  s-0.7321099780664753    s336.370622269264
9  004_3  s-0.3153543481363217  s-0.7321099780911857  s336.37062214563645


In [3]:
# Tree shape vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_high_precision(x, y, angle):
    """Create tree polygon with integer coordinates (scaled by 1e18)"""
    x = Decimal(str(x).replace('s', ''))
    y = Decimal(str(y).replace('s', ''))
    angle = Decimal(str(angle).replace('s', ''))
    
    # Convert angle to radians
    import math
    angle_rad = float(angle) * math.pi / 180
    cos_a = Decimal(str(math.cos(angle_rad)))
    sin_a = Decimal(str(math.sin(angle_rad)))
    
    # Rotate and translate vertices
    vertices = []
    for tx, ty in zip(TX, TY):
        tx_d = Decimal(str(tx))
        ty_d = Decimal(str(ty))
        # Rotate
        rx = tx_d * cos_a - ty_d * sin_a
        ry = tx_d * sin_a + ty_d * cos_a
        # Translate
        px = rx + x
        py = ry + y
        # Scale to integer
        vertices.append((int(px * SCALE), int(py * SCALE)))
    
    return Polygon(vertices)

print("High precision tree function defined")

High precision tree function defined


In [4]:
# Create polygons for N=4
polygons = []
for _, row in n4_trees.iterrows():
    poly = create_tree_high_precision(row['x'], row['y'], row['deg'])
    polygons.append(poly)
    print(f"Tree {row['id']}: area={poly.area / (SCALE**2):.6f}")

print(f"\nCreated {len(polygons)} polygons")

Tree 004_0: area=0.245625
Tree 004_1: area=0.245625
Tree 004_2: area=0.245625
Tree 004_3: area=0.245625

Created 4 polygons


In [5]:
# Check for overlaps between all pairs
print("\nChecking overlaps between all pairs:")
for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        p1, p2 = polygons[i], polygons[j]
        
        intersects = p1.intersects(p2)
        touches = p1.touches(p2)
        overlaps = p1.overlaps(p2)
        
        if intersects:
            intersection = p1.intersection(p2)
            int_area = intersection.area / (SCALE**2)
            print(f"  Trees {i} and {j}: intersects={intersects}, touches={touches}, overlaps={overlaps}, int_area={int_area:.15f}")
            
            if int_area > 1e-20:
                print(f"    *** OVERLAP DETECTED! ***")


Checking overlaps between all pairs:


In [6]:
# Let's also check with Shapely's default precision (no scaling)
from shapely.geometry import Polygon as ShapelyPolygon
from shapely import affinity

def create_tree_shapely(x, y, angle):
    x = float(str(x).replace('s', ''))
    y = float(str(y).replace('s', ''))
    angle = float(str(angle).replace('s', ''))
    
    coords = list(zip(TX, TY))
    poly = ShapelyPolygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

print("\nChecking with Shapely default precision:")
polys_shapely = []
for _, row in n4_trees.iterrows():
    poly = create_tree_shapely(row['x'], row['y'], row['deg'])
    polys_shapely.append(poly)

for i in range(len(polys_shapely)):
    for j in range(i+1, len(polys_shapely)):
        p1, p2 = polys_shapely[i], polys_shapely[j]
        if p1.intersects(p2) and not p1.touches(p2):
            intersection = p1.intersection(p2)
            print(f"  Trees {i} and {j}: OVERLAP! area={intersection.area:.15f}")


Checking with Shapely default precision:


In [7]:
# Let's look at the current best submission (exp_012) for N=4
df_best = pd.read_csv('/home/code/experiments/012_full_snapshot_ensemble/submission.csv')
n4_best = df_best[df_best['id'].str.startswith('004_')].copy()
print("Best submission N=4 trees:")
print(n4_best)

# Check if they're the same
print("\nComparing to current submission:")
for col in ['x', 'y', 'deg']:
    same = (n4_trees[col].values == n4_best[col].values).all()
    print(f"  {col}: same={same}")

Best submission N=4 trees:
      id                     x                     y                  deg
6  004_0  s-0.3247477895908756   s0.1321099780911856   s156.3706221456364
7  004_1   s0.3153543462411342   s0.1321099780664757   s156.3706222692641
8  004_2   s0.3247477895908756  s-0.7321099780664753    s336.370622269264
9  004_3  s-0.3153543481363217  s-0.7321099780911857  s336.37062214563645

Comparing to current submission:
  x: same=True
  y: same=True
  deg: same=True


In [8]:
# The issue is that exp_014 used exp_012 as baseline and found 0 improvements
# So the submission should be identical to exp_012
# But exp_012 was submitted successfully (exp_010 LB: 70.3410)
# Let me check if there's a difference in the submission files

import hashlib

def file_hash(path):
    with open(path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

print("File hashes:")
print(f"  exp_012: {file_hash('/home/code/experiments/012_full_snapshot_ensemble/submission.csv')}")
print(f"  exp_014: {file_hash('/home/code/experiments/014_nfp_local_search/submission.csv')}")
print(f"  /home/submission: {file_hash('/home/submission/submission.csv')}")

File hashes:
  exp_012: 7004d09e43c00a6a752418c3dc04bb75
  exp_014: 7004d09e43c00a6a752418c3dc04bb75
  /home/submission: a819f432e4198222696484964969c12e


In [9]:
# Check if the files are byte-identical
import filecmp

print("\nFile comparison:")
print(f"  exp_012 vs exp_014: {filecmp.cmp('/home/code/experiments/012_full_snapshot_ensemble/submission.csv', '/home/code/experiments/014_nfp_local_search/submission.csv')}")
print(f"  exp_014 vs /home/submission: {filecmp.cmp('/home/code/experiments/014_nfp_local_search/submission.csv', '/home/submission/submission.csv')}")


File comparison:
  exp_012 vs exp_014: True
  exp_014 vs /home/submission: False


In [10]:
# Let's check if exp_012 submission has any issues with N=4
# First, let's see what the original source was for N=4

# Load exp_012 and check N=4
df_012 = pd.read_csv('/home/code/experiments/012_full_snapshot_ensemble/submission.csv')
n4_012 = df_012[df_012['id'].str.startswith('004_')].copy()

print("exp_012 N=4 trees:")
for _, row in n4_012.iterrows():
    print(f"  {row['id']}: x={row['x']}, y={row['y']}, deg={row['deg']}")

# Create polygons and check
polys_012 = [create_tree_shapely(row['x'], row['y'], row['deg']) for _, row in n4_012.iterrows()]

print("\nOverlap check for exp_012 N=4:")
for i in range(len(polys_012)):
    for j in range(i+1, len(polys_012)):
        p1, p2 = polys_012[i], polys_012[j]
        if p1.intersects(p2):
            intersection = p1.intersection(p2)
            if not p1.touches(p2):
                print(f"  Trees {i} and {j}: OVERLAP! area={intersection.area:.15e}")
            else:
                print(f"  Trees {i} and {j}: touches only")

exp_012 N=4 trees:
  004_0: x=s-0.3247477895908756, y=s0.1321099780911856, deg=s156.3706221456364
  004_1: x=s0.3153543462411342, y=s0.1321099780664757, deg=s156.3706222692641
  004_2: x=s0.3247477895908756, y=s-0.7321099780664753, deg=s336.370622269264
  004_3: x=s-0.3153543481363217, y=s-0.7321099780911857, deg=s336.37062214563645

Overlap check for exp_012 N=4:


In [11]:
# Let's check what the original santa-2025.csv has for N=4
df_santa = pd.read_csv('/home/code/kaggle_datasets/santa-2025.csv')
n4_santa = df_santa[df_santa['id'].str.startswith('004_')].copy()

print("santa-2025.csv N=4 trees:")
for _, row in n4_santa.iterrows():
    print(f"  {row['id']}: x={row['x']}, y={row['y']}, deg={row['deg']}")

# Create polygons and check
polys_santa = [create_tree_shapely(row['x'], row['y'], row['deg']) for _, row in n4_santa.iterrows()]

print("\nOverlap check for santa-2025.csv N=4:")
for i in range(len(polys_santa)):
    for j in range(i+1, len(polys_santa)):
        p1, p2 = polys_santa[i], polys_santa[j]
        if p1.intersects(p2):
            intersection = p1.intersection(p2)
            if not p1.touches(p2):
                print(f"  Trees {i} and {j}: OVERLAP! area={intersection.area:.15e}")
            else:
                print(f"  Trees {i} and {j}: touches only")

santa-2025.csv N=4 trees:
  004_0: x=s-0.32474778959087608, y=s0.13210997809118477, deg=s156.37062214563639
  004_1: x=s0.31535434624113440, y=s0.13210997806647518, deg=s156.37062226926409
  004_2: x=s0.32474778959087658, y=s-0.73210997806647471, deg=s336.37062226926406
  004_3: x=s-0.31535434813632124, y=s-0.73210997809118594, deg=s336.37062214563645

Overlap check for santa-2025.csv N=4:


In [12]:
# Create a new ensemble with all available sources
import os
import glob

# Find all CSV files
all_csvs = glob.glob('/home/code/kaggle_datasets/**/*.csv', recursive=True)
print(f"Found {len(all_csvs)} CSV files")

# Also include snapshots
snapshot_csvs = glob.glob('/home/nonroot/snapshots/**/*.csv', recursive=True)
print(f"Found {len(snapshot_csvs)} snapshot files")

# Filter to only valid submission files (should have 20100 rows)
valid_csvs = []
for csv_path in all_csvs + snapshot_csvs:
    try:
        df = pd.read_csv(csv_path, nrows=5)
        if 'id' in df.columns and 'x' in df.columns:
            valid_csvs.append(csv_path)
    except:
        pass

print(f"Found {len(valid_csvs)} valid CSV files")

Found 383 CSV files
Found 4105 snapshot files


Found 4482 valid CSV files


In [13]:
# Load current best and compute per-N scores
from shapely.ops import unary_union

def compute_score_for_n(trees):
    """Compute bounding box score for a list of trees"""
    if len(trees) == 0:
        return 0
    polygons = [create_tree_shapely(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
    return (side ** 2) / len(trees)

# Load current best
df_best = pd.read_csv('/home/code/experiments/012_full_snapshot_ensemble/submission.csv')
df_best['N'] = df_best['id'].str.split('_').str[0].astype(int)

best_per_n = {}
best_scores = {}

for n, g in df_best.groupby('N'):
    trees = [{'x': row['x'], 'y': row['y'], 'deg': row['deg']} for _, row in g.iterrows()]
    best_per_n[n] = trees
    best_scores[n] = compute_score_for_n(trees)

baseline_total = sum(best_scores.values())
print(f"Current best total score: {baseline_total:.6f}")

Current best total score: 70.340917


In [14]:
# Process each CSV and find improvements
improvements = []
sources_used = set()

# Sample a subset for speed
import random
sample_csvs = random.sample(valid_csvs, min(200, len(valid_csvs)))

for csv_path in sample_csvs:
    try:
        df = pd.read_csv(csv_path)
        if len(df) != 20100:
            continue
        
        df['N'] = df['id'].str.split('_').str[0].astype(int)
        
        for n, g in df.groupby('N'):
            trees = [{'x': row['x'], 'y': row['y'], 'deg': row['deg']} for _, row in g.iterrows()]
            
            # Validate no overlaps
            if not validate_no_overlap_strict(trees):
                continue
            
            score = compute_score_for_n(trees)
            
            if score < best_scores[n] - 1e-9:
                improvement = best_scores[n] - score
                improvements.append((n, improvement, csv_path))
                best_per_n[n] = trees
                best_scores[n] = score
                sources_used.add(csv_path)
                print(f"N={n}: +{improvement:.6f} from {os.path.basename(csv_path)}")
    except Exception as e:
        pass

print(f"\\nTotal improvements: {len(improvements)}")
print(f"Sources used: {len(sources_used)}")

\nTotal improvements: 0
Sources used: 0


In [15]:
# Check the new sources specifically
new_sources = [
    '/home/code/kaggle_datasets/telegram_public/71.97.csv',
    '/home/code/kaggle_datasets/telegram_public/72.49.csv',
    '/home/code/kaggle_datasets/egortrushin_improved_sa_new/submission.csv',
    '/home/code/kaggle_datasets/jazivxt_why_not/submission.csv',
    '/home/code/kaggle_datasets/eyestrain_blending_new/submission.csv',
]

for csv_path in new_sources:
    try:
        df = pd.read_csv(csv_path)
        if len(df) != 20100:
            print(f"{os.path.basename(csv_path)}: Invalid length {len(df)}")
            continue
        
        df['N'] = df['id'].str.split('_').str[0].astype(int)
        
        # Compute total score
        total_score = 0
        for n, g in df.groupby('N'):
            trees = [{'x': row['x'], 'y': row['y'], 'deg': row['deg']} for _, row in g.iterrows()]
            total_score += compute_score_for_n(trees)
        
        print(f"{os.path.basename(csv_path)}: {total_score:.6f} (vs best {baseline_total:.6f})")
    except Exception as e:
        print(f"{os.path.basename(csv_path)}: Error - {e}")

71.97.csv: 71.972027 (vs best 70.340917)


72.49.csv: 72.495739 (vs best 70.340917)


submission.csv: 72.245177 (vs best 70.340917)


submission.csv: 70.332155 (vs best 70.340917)


submission.csv: 71.727773 (vs best 70.340917)


In [ ]:
# The jazivxt_why_not submission is better! Let's check it for overlaps and ensemble
df_jazivxt = pd.read_csv('/home/code/kaggle_datasets/jazivxt_why_not/submission.csv')
df_jazivxt['N'] = df_jazivxt['id'].str.split('_').str[0].astype(int)

print("Checking jazivxt_why_not for overlaps and improvements...")

jazivxt_improvements = []
jazivxt_overlaps = []

for n, g in df_jazivxt.groupby('N'):
    trees = [{'x': row['x'], 'y': row['y'], 'deg': row['deg']} for _, row in g.iterrows()]
    
    # Check overlaps
    if not validate_no_overlap_strict(trees):
        jazivxt_overlaps.append(n)
        continue
    
    score = compute_score_for_n(trees)
    
    if score < best_scores[n] - 1e-9:
        improvement = best_scores[n] - score
        jazivxt_improvements.append((n, improvement))
        print(f"N={n}: +{improvement:.6f}")

print(f"\\nTotal improvements: {len(jazivxt_improvements)}")
print(f"N values with overlaps: {len(jazivxt_overlaps)}")
if jazivxt_overlaps:
    print(f"Overlap N values: {jazivxt_overlaps[:20]}...")

In [17]:
# Define validation function
def validate_no_overlap_strict(trees_data):
    if len(trees_data) <= 1:
        return True
    
    polygons = []
    for t in trees_data:
        x = Decimal(str(t['x']).replace('s', ''))
        y = Decimal(str(t['y']).replace('s', ''))
        angle = Decimal(str(t['deg']).replace('s', ''))
        
        import math
        angle_rad = float(angle) * math.pi / 180
        cos_a = Decimal(str(math.cos(angle_rad)))
        sin_a = Decimal(str(math.sin(angle_rad)))
        
        vertices = []
        for tx, ty in zip(TX, TY):
            tx_d = Decimal(str(tx))
            ty_d = Decimal(str(ty))
            rx = tx_d * cos_a - ty_d * sin_a
            ry = tx_d * sin_a + ty_d * cos_a
            px = rx + x
            py = ry + y
            vertices.append((int(px * SCALE), int(py * SCALE)))
        
        polygons.append(Polygon(vertices))
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return False
    return True

print("Validation function defined")

Validation function defined


In [ ]:
# The issue might be that our submission file got corrupted during the NFP process
# Let's use the exp_012 submission directly since it was validated

import shutil
shutil.copy('/home/code/experiments/012_full_snapshot_ensemble/submission.csv', '/home/submission/submission.csv')
print("Copied exp_012 submission to /home/submission/submission.csv")

# Verify
print(f"\nVerification:")
print(f"  /home/submission hash: {file_hash('/home/submission/submission.csv')}")
print(f"  exp_012 hash: {file_hash('/home/code/experiments/012_full_snapshot_ensemble/submission.csv')}")